In [1]:
import numpy as np
from dataloader import *
from models import *
from configs import cfg
from torch.nn.functional import softmax
from nltk.translate import bleu_score
import time

In [2]:
cfg['hidden_dim'] = 128 # hidden dimension for LSTM
cfg['layers'] = 2 # number of layers of LSTM
cfg['dropout'] = 0.01 # dropout rate between two layers of LSTM; useful only when layers > 1; between 0 and 1
cfg['bidirectional'] = False # True or False; True means using a bidirectional LSTM
cfg['batch_size'] = 100 # batch size of input
cfg['learning_rate'] = 5e-5 # learning rate to be used
cfg['L2_penalty'] = 1e-5 # weighting constant for L2 regularization term; this is a parameter when you define optimizer
cfg['epochs'] = 5 # number of epochs for which the model is trained
cfg['train_split'] = 0.8
cfg['model_type'] = 'gru'
cfg['device'] = 'cuda'

In [3]:
training_data = np.load('/datasets/home/32/032/sugao/CSE190/pa4/train.npy')

In [4]:
models_dir = './models/5-5_1-5/'
val_dir = models_dir + 'val.npy'

device = torch.device('cuda')
model = BeerGruMind(cfg, True)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=cfg['learning_rate'], weight_decay=cfg['L2_penalty'])
train_indices, valid_indices = train_valid_split(len(training_data), cfg['train_split'])
np.save(val_dir, np.array(valid_indices))
print('ready')

ready


In [ ]:
train_losses = []
valid_losses = []
for epoch in range(cfg['epochs']):
    tloader = DataLoader(training_data, train_indices, cfg)
    vloader = DataLoader(training_data, valid_indices, cfg)

    count = 0
    avg_loss = 0
    losss = []
    while tloader.has_next():
        train, label = tloader.get_next()

        model.zero_grad()
        
        y = model(train)

        y = y.permute(0, 2, 1)

        loss = criterion(y, label)
        loss.backward()
        optimizer.step()

        count += 1
        avg_loss += loss.item()
        if count % 500 == 0:
            print("count = %d, training loss = %.5f" %(count, avg_loss / 500) )
            save_model(model, models_dir + 'e%d_c%d_ls%.3f' % (epoch, count, avg_loss/500))
            losss.append(avg_loss)
            avg_loss = 0
    train_losses.append(losss)
    
    count = 0
    avg_loss = 0
    with torch.no_grad():
        while vloader.has_next():
            train, label = vloader.get_next()
            y = model(train)
            y = y.permute(0, 2, 1)
            loss = criterion(y, label)
            count += 1
            avg_loss += loss.item()
    print('validation loss:', avg_loss / count)
    print('epoch finished:', epoch + 1)
    valid_losses.append(avg_loss/count)

np.save(models+'val_loss.npy', np.array(valid_losses))
np.save(models+'train_loss.npy', np.array(train_losses))

count = 500, training loss = 1.41242
count = 1000, training loss = 0.99226
count = 1500, training loss = 0.94010
count = 2000, training loss = 0.86639
count = 2500, training loss = 0.78229
count = 3000, training loss = 0.74915
count = 3500, training loss = 0.71428
count = 4000, training loss = 0.67512
count = 4500, training loss = 0.65873
count = 5000, training loss = 0.62687
